In [1]:
#importer les packages
import os
import pandas as pd
from tqdm.notebook import tqdm
tqdm.pandas()

from utils.correcting_features import corrige_all, corrige_etabli2, corrige_rgp2, corrige_rgp3, corrige_op_ing
from utils.load_corrections import get_all_correctifs_from_google, load_all_correctifs
from utils.traitements import importtab, gentab

In [2]:
# ATTENTION, avant de commencer, s'assurer d'avoir un dossier "INPUT" 
# avec les données au format parquet pour chaque année et un dossier
# POST_GENTAB pour acceuillir les données traitées (attention c'est assez volumineux)

# Modifier la page format du googlesheet si il y a de nouveaux formats cette année
# Modifier également la page D_EPE du googlesheet ESR pour ajouter les EPE de la nouvelle année

In [3]:
DATA_PATH=os.getcwd()

In [4]:
url_atlas='https://docs.google.com/spreadsheet/ccc?key=11NFXSIg6gQMCsMa8zWQQyypvvYBEmfyJfF2yytXqgMk&output=xls'
url_esr='https://docs.google.com/spreadsheet/ccc?key=1FwPq5Qw7Gbgj_sBD6Za4dfDDk6ydozQ99TyRjLkW5d8&output=xls'

VARS_atlas=['COM_U','COM_M', 'ETABLI', 'MINISTER', 'DC', 'NATION', 'ETABLI2', 'FORMAT','CURSUS_LMD', 'DISCIPLI','RGP2', 'RGP3','OP_ING','SECRET']
VARS_esr=['O_DUTBUT','J_LMDDONT','DISCIPLINES_SISE','FORMAT','LES_COMMUNES','D_EPE','C_ETABLISSEMENTS']

Charger les données des googlesheets depuis google en local au format .json dans le dossier (pas besoin de le faire à chaque fois, à faire si c'est la première fois OU lors d'une modification sur le.s googlesheet.s)

In [5]:
get_all_correctifs_from_google(url_atlas,VARS_atlas,'_atlas')
get_all_correctifs_from_google(url_esr,VARS_esr,'_esr')

Charger les 2 googlessheets dans le notebook

In [6]:
CORRECTIFS_dict_esr = load_all_correctifs('_esr')
CORRECTIFS_dict = load_all_correctifs('_atlas')

In [7]:
formats=pd.DataFrame(CORRECTIFS_dict_esr['FORMAT']).drop_duplicates()

On charge bien la rentrée scolaire de l'année en cours

In [7]:
rentree_sco=2023

On charge les données correspondantes

In [8]:
df=pd.read_parquet(f"./INPUT/ts{str(rentree_sco)[2:]}d_tot.parquet", engine='fastparquet')

In [9]:
df.columns=[x.upper() for x in df.columns]

In [10]:
#verifier que HCPGE est bien rempli
if 'HCPGE' in list(df.columns):
    print(df.HCPGE.value_counts())

In [11]:
#voir les effectifs sans les doubles comptes
if 'HSIFA' in list(df.columns):
    print(df.EFFTOT.sum()-df.loc[(df['HSIFA']==0.0),'EFFTOT'].sum())
    print(df.loc[(df['HSIFA']==0.0),'EFFTOT'].sum())

On applique la fonction importtab, qui va corriger les features une par une de cette année

In [ ]:
df_up=importtab(df,CORRECTIFS_dict,CORRECTIFS_dict_esr,corrige_etabli2,corrige_all,rentree_sco)

In [ ]:
# test EFFTOT si ils sont toujours egaux sans HSIFA
if 'HSIFA' in list(df.columns):
    print(df_up['HSIFA'].value_counts())
    print(df_up.EFFTOT.sum()-df_up.loc[(df_up['HSIFA']==0.0),'EFFTOT'].sum()==df.EFFTOT.sum()-df.loc[(df['HSIFA']==0.0),'EFFTOT'].sum())

In [ ]:
# test si nouveaux formats cette année
[x for x in list(df_up.FORMAT.drop_duplicates()) if x not in list(pd.DataFrame(CORRECTIFS_dict_esr['FORMAT'])['FORMAT'].drop_duplicates())]

In [15]:
#fonction gentab

In [ ]:
df_up2= gentab(df_up, rentree_sco, CORRECTIFS_dict, CORRECTIFS_dict_esr, corrige_rgp2, corrige_rgp3, corrige_op_ing)

On regarde si les effectifs des apprentis sont présents

In [ ]:
if 'EFF_STS_APP' in df_up2.columns:
    print(df_up2.EFF_STS_APP.value_counts())

Pour chaque année, on enregistre les données traitées

In [18]:
df_up2.to_json(f"./POST_GENTAB/atlas{str(rentree_sco)}.json")